In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
from decision import haversine
from sklearn.cross_validation import train_test_split
import pickle
    
%matplotlib inline

In [2]:
def predict_batch(traj_tree, trajectories):
    return [predict_one(traj_tree, trajectory) for trajectory in trajectories] 
        
def predict_one(traj_tree, trajectory):
    current_node = traj_tree.root
        
    for idx, point in enumerate(trajectory):
        if len(trajectory) > (idx + 1):
            if (current_node.left == None) and (current_node.right == None):
                prediction = current_node.prediction
                break
                    
            elif (current_node.left == None) and (current_node.right != None):
                current_node = current_node.right
                    
            elif (current_node.left != None) and (current_node.right == None):
                current_node = current_node.left
                    
            else:
                if haversine(point, current_node.decision_point) < current_node.radius:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
                        
        else:
            prediction = current_node.prediction
            break
                
        #return [float(prediction[0]), float(prediction[1])]
    return prediction

In [3]:
test_trajs = pd.read_csv('test.csv',
                        #index_col=,
                        usecols = ['TRIP_ID', 'POLYLINE'],
                        converters={'POLYLINE': lambda x: json.loads(x)})

In [4]:
tree = pickle.load( open( "tree_500_data.p", "rb" ) )

In [5]:
lats = []
lons = []

for i in range(len(test_trajs)):
    prediction = predict_one(tree, test_trajs['POLYLINE'][i])
    lats.append(prediction[1])
    lons.append(prediction[0])

In [6]:
test_trajs['LATITUDE'] = np.array(lats)
test_trajs['LONGITUDE'] = np.array(lons)

In [7]:
df_pred = test_trajs.drop('POLYLINE', 1)

In [8]:
df_pred.to_csv('test_submission_500.csv', index = False)